<a href="https://colab.research.google.com/github/shahid9455/AI_MINDS_GPTPLUS/blob/main/AI%20MINDS%20CHATPLUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio gtts graphviz

In [5]:
import gradio as gr
import requests
from PIL import Image
from io import BytesIO
import tempfile
from gtts import gTTS
import re
import graphviz
import os

# Load the API key from environment variables for security
API_KEY = 'c11088e2e97542b09e8aa2e9b93311d9'

# Define API endpoints
IMAGE_API_URL = 'https://api.aimlapi.com/images/generations'
CHAT_API_URL = 'https://api.aimlapi.com/chat/completions'

# List of available chat models
CHAT_MODELS = [
    "meta-llama/Meta-Llama-3-8B-Instruct-Lite",
    "meta-llama/Meta-Llama-3-70B-Instruct-Lite",
    "meta-llama/Meta-Llama-3-70B-Instruct-Turbo",
    "meta-llama/Meta-Llama-3-8B-Instruct-Turbo",
    "gpt-4o",
    "gpt-4o-mini-2024-07-18"
]

# Load supported languages from a file
def load_languages(file_path='languages.txt'):
    languages = {}
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                if line.strip():
                    language, code = line.strip().split(': ')
                    languages[language] = code
    except FileNotFoundError:
        print(f"Error: {file_path} not found.")
    return languages

languages = load_languages()

def get_answer_content(language_name, question, model_name, category, max_chars, max_lines):
    language_code = languages.get(language_name, 'en')
    headers = {
        'Authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json'
    }
    data = {
        "model": model_name,
        "messages": [
            {
                "role": "user",
                "content": f"Respond in {language_name} for category '{category}': {question}"
            }
        ],
        "max_tokens": 1500,
        "stream": False
    }
    try:
        response = requests.post(CHAT_API_URL, headers=headers, json=data)
        response.raise_for_status()
        answer_content = response.json()['choices'][0]['message']['content']

        if category in ["Documentation", "Research"]:
            answer_content = answer_content[:1500]

        # Truncate to max_chars
        if max_chars:
            answer_content = answer_content[:int(max_chars)]

        # Ensure the output ends with a complete sentence
        if max_chars:
            truncated_length = int(max_chars)
            if truncated_length < len(answer_content):
                # Find the last sentence-ending punctuation within the truncated length
                last_punctuation_index = max(
                    answer_content.rfind(p) for p in ".!?"
                )
                if last_punctuation_index > -1 and last_punctuation_index <= truncated_length:
                    answer_content = answer_content[:last_punctuation_index + 1]
                else:
                    # If no punctuation is found or it's outside the limit, truncate at the limit
                    answer_content = answer_content[:truncated_length]

        # Limit by max_lines if specified
        if max_lines:
            answer_content = "\n".join(answer_content.splitlines()[:int(max_lines)])

        # Remove unwanted introductory lines
        lines = answer_content.splitlines()
        filtered_lines = [line for line in lines if not line.lower().startswith("here's a joke about")]
        filtered_content = "\n".join(filtered_lines)

        return filtered_content
    except requests.RequestException as e:
        return f"An error occurred: {e}"

def preprocess_text(text):
    return re.sub(r'[^\w\s,.!?]', '', text)

def text_to_speech_online(text, lang='en'):
    try:
        cleaned_text = preprocess_text(text)
        tts = gTTS(text=cleaned_text, lang=lang, slow=False)
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as temp_file:
            tts.save(temp_file.name)
            return temp_file.name
    except Exception as e:
        print(f"Text-to-speech failed: {e}")
        return None

def generate_image(prompt, model_name):
    headers = {"Authorization": f"Bearer {API_KEY}"}
    payload = {"prompt": prompt, "model": model_name}
    try:
        response = requests.post(IMAGE_API_URL, headers=headers, json=payload)
        response.raise_for_status()
        output = response.json()
        if "images" in output and output["images"]:
            image_url = output["images"][0]["url"]
            img_data = requests.get(image_url).content
            image = Image.open(BytesIO(img_data))
            return image
        else:
            print("Unexpected response structure:", output)
            return Image.new('RGB', (512, 512), color=(255, 0, 0))
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return Image.new('RGB', (512, 512), color=(255, 0, 0))
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return Image.new('RGB', (512, 512), color=(255, 0, 0))

def wrap_text(text, width=30):
    words = text.split()
    lines, current_line, current_length = [], [], 0

    for word in words:
        if current_length + len(word) <= width:
            current_line.append(word)
            current_length += len(word) + 1
        else:
            lines.append(" ".join(current_line))
            current_line = [word]
            current_length = len(word) + 1

    lines.append(" ".join(current_line))
    return "\n".join(lines)

def generate_workflow_diagram(steps):
    dot = graphviz.Digraph(format='png')
    dot.attr(rankdir='TB', size='10,10', nodesep='0.5', ranksep='0.5', dpi='300')

    steps = steps.strip().split("\n")
    if not steps or steps == [""]:
        return None

    for i, step in enumerate(steps):
        step = wrap_text(step.strip(), width=30)
        if step:
            dot.node(str(i), step, shape='box', width='2.0', height='0.5', fontsize='12')
            if i > 0:
                dot.edge(str(i - 1), str(i))

    with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as temp_file:
        dot.render(temp_file.name)
        return temp_file.name + '.png'

def on_button_click(language_name, question, model_name, category, max_chars, max_lines):
    if not question.strip():
        return "Please enter a question.", None

    if category == "default":
        category = "Post"

    answer = get_answer_content(language_name, question, model_name, category, max_chars, max_lines)
    audio_file = text_to_speech_online(answer, languages.get(language_name, 'en'))

    return f"You: {question}\n\nAI MINDS:\n\n{answer}", audio_file

def on_image_button_click(prompt, model_name):
    return generate_image(prompt, model_name)

def on_workflow_button_click(steps):
    return generate_workflow_diagram(steps)

def clear_all():
    return None, None, None, None, None

# Define Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# AI_MINDS ChatPlus")

    with gr.Tabs():
        with gr.Tab("Chat"):
            gr.Markdown("## Chat Section")
            with gr.Row():
                language_dropdown = gr.Dropdown(choices=list(languages.keys()), label="Select Language", value="English")
                model_dropdown = gr.Dropdown(choices=CHAT_MODELS, label="Select Chat Model", value="meta-llama/Meta-Llama-3-70B-Instruct-Turbo")
                category_dropdown = gr.Dropdown(choices=["default","Post", "Documentation", "Research", "Generation"], label="Select Category", value="default")
                max_chars_input = gr.Number(label="Max Characters (Optional)", value=None, step=1, precision=0)
                max_lines_input = gr.Number(label="Max Lines (Optional)", value=None, step=1, precision=0)

            with gr.Row():
                with gr.Column(scale=1):
                    question_input = gr.Textbox(label="Your Question", placeholder='Ask a question...', lines=2)
                    generate_button = gr.Button("Ask")
                    small_audio_output = gr.Audio(label="Voice Output", type="filepath", visible=True, interactive=False)
                    clear_button = gr.Button("Clear")

                with gr.Column(scale=2):
                    content_output = gr.Markdown(label="Chat Output")

        with gr.Tab("Image"):
            gr.Markdown("## Image Generation Section")
            image_prompt = gr.Textbox(label="Image Prompt", placeholder='Enter an image prompt...')
            image_model_dropdown = gr.Dropdown(choices=["flux-realism", "stable-diffusion-v3-medium"], label="Select Image Model", value="flux-realism")
            generated_image = gr.Image(label="Generated Image", type="pil")
            image_generate_button = gr.Button("Generate Image")

        with gr.Tab("Flowchart"):
            gr.Markdown("## Workflow Diagram Generator")
            workflow_input = gr.Textbox(lines=10, placeholder="Enter workflow steps, one per line.", label="Workflow Steps")
            generate_workflow_button = gr.Button("Generate Diagram")
            diagram_output = gr.Image(label="Generated Workflow Diagram")

    # Define button actions
    generate_button.click(on_button_click, [language_dropdown, question_input, model_dropdown, category_dropdown, max_chars_input, max_lines_input], [content_output, small_audio_output])
    image_generate_button.click(on_image_button_click, [image_prompt, image_model_dropdown], [generated_image])
    generate_workflow_button.click(on_workflow_button_click, [workflow_input], [diagram_output])

    clear_button.click(fn=clear_all, inputs=[], outputs=[question_input, content_output, generated_image, diagram_output, small_audio_output])

# Launch the Gradio app
if __name__ == "__main__":
    demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9611b357d9377e680f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
